This notebooks takes the model trained in `Colabs/GroceryDataset_Model_Training.ipynb` notebook and runs inference with it to determine how many products are likely to be present inside a given shelf image. 

## Inital setup

In [ ]:
# Which GPU?
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [1]:
# Install TFOD API (TF 1)
%tensorflow_version 1.x
import tensorflow as tf 
print(tf.__version__)

!git clone https://github.com/tensorflow/models.git

% cd models/research
!pip install --upgrade pip
# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf1/setup.py .
!python -m pip install --use-feature=2020-resolver .

TensorFlow 1.x selected.
1.15.2
Cloning into 'models'...
remote: Enumerating objects: 60399, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 60399 (delta 81), reused 141 (delta 59), pack-reused 60221
Receiving objects: 100% (60399/60399), 573.89 MiB | 31.87 MiB/s, done.
Resolving deltas: 100% (41973/41973), done.
/content/models/research
     |████████████████████████████████| 1.6 MB 5.0 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this a

## Gather trained model and test data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
!pwd

/content/models/research


In [10]:
!cp -r /content/drive/MyDrive/GroceryShelf/Product-Detection-From-Grocery-Shelf/models/research/object_detection/inference_graph/effdet/saved_model/*.pb .
!ls -lh *.pb

-rw------- 1 root root 23M Aug 21 13:04 saved_model.pb


In [12]:
%cd /content/drive/MyDrive/GroceryShelf/Product-Detection-From-Grocery-Shelf/
!ls -lh ../Dataset/GroceryDataset_part1/ShelfImages/test | head -10

/content/drive/MyDrive/GroceryShelf/Product-Detection-From-Grocery-Shelf
total 101M
-rw------- 1 root root 1.6M Oct 23  2019 C1_P02_N1_S5_1.JPG
-rw------- 1 root root 2.3M Oct 23  2019 C1_P02_N2_S2_1.JPG
-rw------- 1 root root 2.3M Oct 23  2019 C1_P02_N2_S3_1.JPG
-rw------- 1 root root 1.3M Oct 23  2019 C1_P03_N1_S2_1.JPG
-rw------- 1 root root 1.6M Oct 23  2019 C1_P03_N1_S3_1.JPG
-rw------- 1 root root 2.4M Oct 23  2019 C1_P03_N1_S4_1.JPG
-rw------- 1 root root 1.4M Oct 23  2019 C1_P03_N1_S4_2.JPG
-rw------- 1 root root 1.1M Oct 23  2019 C1_P03_N2_S2_1.JPG
-rw------- 1 root root 1.7M Oct 23  2019 C1_P03_N2_S3_1.JPG


## Other imports

In [13]:
from imutils import paths
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import json 

## Image parsing utility

In [14]:
def parse_image(image_path: str) -> np.ndarray:
    """Reads an image and adds a batch dimension."""
    image = plt.imread(image_path).astype(np.uint8)
    image = np.expand_dims(image, 0)
    return image

## Load test image paths

In [15]:
test_image_paths = list(paths.list_images("../Dataset/GroceryDataset_part1/ShelfImages/test"))
test_image_paths[:5]

['../Dataset/GroceryDataset_part1/ShelfImages/test/C3_P01_N1_S5_1.JPG',
 '../Dataset/GroceryDataset_part1/ShelfImages/test/C4_P03_N1_S3_1.JPG',
 '../Dataset/GroceryDataset_part1/ShelfImages/test/C4_P04_N4_S2_1.JPG',
 '../Dataset/GroceryDataset_part1/ShelfImages/test/C4_P08_N1_S4_1.JPG',
 '../Dataset/GroceryDataset_part1/ShelfImages/test/C1_P03_N2_S2_1.JPG']

## Load detection graph

In [22]:
import tensorflow as tf
import sys
from tensorflow.python.platform import gfile
from tensorflow.core.protobuf import saved_model_pb2
from tensorflow.python.util import compat

with tf.Session() as sess:
    model_filename ='./models/research/object_detection/inference_graph/effdet/saved_model/saved_model.pb'
    with gfile.FastGFile(model_filename, 'rb') as f:
        data = compat.as_bytes(f.read())
        sm = saved_model_pb2.SavedModel()
        sm.ParseFromString(data)
        g_in = tf.import_graph_def(sm.meta_graphs[0].graph_def)

## Inference utility

In [26]:
def run_inference_for_single_image(image, graph, min_threshold=0.6):
    """Runs detection graph on an image and parses the results."""
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                "num_detections", "detection_boxes", "detection_scores",
                "detection_classes"]:
                tensor_name = key + ":0"
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
                image_tensor = 'image_tensor:0'

            # Run inference
            output_dict = sess.run(tensor_dict,
                                    feed_dict={image_tensor: image})

    # Post-process the results
    output_dict["detection_scores"] = output_dict["detection_scores"][0]
    mask = output_dict["detection_scores"] > min_threshold

    return output_dict["detection_scores"][mask]

## Run bulk inference and prepare JSON file

In [ ]:
tf.compat.v1.disable_eager_execution()

image_to_products = {}
for image_path in tqdm(test_image_paths):
    image_name = image_path.split("/")[-1]
    image = parse_image(image_path)
    num_products = len(run_inference_for_single_image(image, detection_graph))
    image_to_products[image_name] = num_products

json_string = json.dumps(image_to_products, indent=4) 
with open("image2products.json", "w") as outfile: 
    outfile.write(json_string) 

In [ ]:
!head -10 image2products.json

{
    "C2_P03_N2_S3_1.JPG": 22,
    "C1_P10_N1_S5_1.JPG": 51,
    "C3_P01_N2_S3_2.JPG": 17,
    "C4_P03_N1_S3_1.JPG": 33,
    "C2_P04_N3_S2_1.JPG": 21,
    "C3_P03_N2_S4_1.JPG": 49,
    "C3_P04_N1_S5_1.JPG": 40,
    "C4_P08_N2_S2_1.JPG": 24,
    "C4_P03_N1_S4_1.JPG": 45,


## References
* https://github.com/anirbankonar123/CorrosionDetector/blob/master/rust_localization.ipynb